# 1 Leiðarval hjá Strætó
Í þessu verkefni á að búa til leitarvél fyrir Strætó, notast verður við raungögn frá Strætó
sem er að finna á
https://straeto.is/um-straeto/opin-rafraen-gogn.
Til einföldunar munuð þið nota skrána gtfs.zip sem er að finna á Canvas síðu námskeiðsins til að tryggja að
allir noti sömu útgáfu.


## 2.1 Inntak
Inntakið er tímatafla strætó, listi yfir leiðir, vagna og stoppistöðvar. Við skilgreinum netið
G á eftirfarandi hátt. Hver einasta stoppistöð verður hnútur.
Ef vagn ekur á milli stoppistöðva x og y á tímum t1 og t2 með engum öðrum stoppum á
milli þá búum við til nýjan hnút v með leggina (x, v), (v, y) og merkjum þá með tímunum t_1 og t_2. Hnúturinn v geymir upplýsingar um hvaða vagni sá hnútur tilheyrir, athugið þessi
hnútur verður ekki endurnýttur.
Fyrir leiðina sem er sýnd á myndinni þá samsvarar hún fimm hnútum. Hnútar 1, 3, 5
eru stoppistöðvarnar Hlemmur B, Sólfarið og Harpa. Hnútur 2 tengir saman Hlemm B og
Sólfarið og leggirnir hafa tímann 10:22 og 10:25. Hnútur 4 tengir saman Sólfarið og Hörpu,
fyrri leggurinn hefur tímann 10:25 (ekki sýnilegt á mynd) og sá seinni 10:26. Þetta er dæmi
um löglegan veg þar sem allir leggir samsvara vögnum sem við getum tekið og tímarnir
stemma, þ.e. eru í vaxandi röð

## 2.2 Reiknirit
Einfaldasta tilfellið sem við viljum leysa þá fá um við par af stoppistöðvum og tíma, (x, y, t)
og úttakið er leið í strætókerfinu frá x til y sem fer frá x á tíma t′ ≥ t og lágmarkar
komutímann á áfangastað. Ef við skiptum um strætisvagn þá samsvarar það því að "bíða"á
hnútnum sem er á stoppistöð og taka legg sem tilheyrir öðrum vagni heldur en þeim sem
við komum með.
Til að leysa þetta verkefni er hægt að nota reiknirit Dijkstra með einni breytingu, í stað
þess að slaka (RELAX aðferðin) á leggjum með því að leggja saman vigtir þá notum við
komutímann á leggnum ef leggurinn er löglegur, þ.e. ef (u, v) er leggurinn sem við skoðum
með tímann t þá setjum við d[v] = min(d[v], t) ef t ≥ d[u].


# 2.3 Verkþættir
## 2.3.1 Þáttun (⋆)
Gögnin fyrir strætó eru gefin á GTFS formi, lýsingu á GTFS er að finna á https://gtfs.
org/schedule/reference/. Til að búa til netið þarf að þátta (e. parse) gögnin, þ.e. lesa
þau inn og tengja saman á skynsamlegan hátt.

Til einföldunar ætlum við eingöngu að nota eftirfarandi hluta af gögnunum

agency_id=1 eingöngu leiðir sem tilheyra Strætó BS merktar ST.*

Leiðirnar eru ólíkar milli vikudaga, gefum okkur að við séum að vinna með virka daga
og sleppum næturstrætó (leiðir 101-106). Það má líka sleppa helgidögum, þ.e. sem er
skilgreint í calendar_dates.csv skráni.

Sumar leiðir eru í pöntunarþjónustu, við höldum eingöngu þeim leggjum sem eru með
pickup_type=0 í stop_times.csv.

#### import

In [21]:
import pandas as pd
from datetime import datetime, timedelta
import heapq
import math

#### Hjálparföll

In [22]:
# fall til að finna út hvort við séum með tvö id's með sama nafn
def find_unique_stop_ids(df, stop_name):
    filtered_df = df[df['stop_name'] == stop_name]
    unique_stop_ids = filtered_df['stop_id'].unique()
    return unique_stop_ids

In [31]:
# Lesa inn gögnin
stops_df = pd.read_csv('content/stops.txt')
trips_df = pd.read_csv('content/trips.txt')
stop_times_df = pd.read_csv('content/stop_times.txt')
routes_df = pd.read_csv('content//routes.txt', comment='#')
calendar_dates_df = pd.read_csv('content/calendar_dates.txt')
# næturstrætó
night_buses = ['101', '102', '103', '104', '105', '106']

# Filter -> bara sem byrjar á ST. & agency_id = 1 & er ekki næturstrætó
routes_df = routes_df[(routes_df['route_id'].str.startswith("ST.", na=False)) &
                      (routes_df['agency_id'] == 1) &
                      (~routes_df['route_short_name'].isin(night_buses))]

# Bæta við dálki sem hefur aðeins upplýsingar um vikudaga.
trips_df['day_pattern'] = trips_df['service_id'].apply(lambda x: x.split('_')[1])

# Filter -> bara virkir dagar.
def all_weekdays_no_weekends(pattern):
    weekdays = 'MTWTF'
    return all(day in pattern for day in weekdays if day != '-')
    
filtered_trips_df = trips_df[trips_df['day_pattern'].apply(all_weekdays_no_weekends)]

# Filter -> pickup_type = 0
filtered_stop_times_df = stop_times_df[stop_times_df['pickup_type'] == 0]

# Merge -> filtered_stop_times & stops til að fá nafn á stoppi
stop_times_with_stops_df = pd.merge(filtered_stop_times_df, stops_df, on='stop_id')

# Merge -> stop_times_with_stops & trips til að fá upplýsingar um ferðir
stop_times_with_stops_and_trips_df = pd.merge(stop_times_with_stops_df, filtered_trips_df, on='trip_id')

# Merge -> Henda þessu í öllu í eina klessu
stop_times_with_stops_trips_and_routes_df = pd.merge(stop_times_with_stops_and_trips_df, routes_df[['route_id', 'route_short_name']], on='route_id')

# Merge -> stop_times_with_stops_trips_and_routes & calendar_dates til að fá upplýsingar um dagsetningar
filtered_final_df = pd.merge(stop_times_with_stops_trips_and_routes_df, calendar_dates_df, on='service_id')

# Filter -> bara fyrir daginn 20240411
filtered_final_current_df = filtered_final_df[filtered_final_df['date'] == 20240411] # finna betri leið service id er unique og nýjasta

# Filter -> óþarfa gögn
df_re_filtered = filtered_final_current_df.drop(['stop_headsign', 'trip_short_name', 'exception_type', 'service_id', 'route_id', 'shape_id', 'date', 'day_pattern', 'pickup_type','location_type'], axis=1)

# raða til að geta sett inn rétta liði
df = df_re_filtered.sort_values(['trip_id', 'stop_sequence'])

stop_id_to_name = stops_df.set_index('stop_id')['stop_name'].to_dict()

# Assign 'next_stop_id' and 'next_arrival_time'
df['next_stop_id'] = df.groupby('trip_id')['stop_id'].shift(-1)
df['next_arrival_time'] = df.groupby('trip_id')['arrival_time'].shift(-1)

# Check if the column has been created and has NaN values that need filling
if 'next_arrival_time' in df.columns:
    df['next_arrival_time'].fillna('End of trip', inplace=True)
else:
    print("Error: 'next_arrival_time' column not found")

# Convert 'next_stop_id' to 'next_stop_name' using the mapping
df['next_stop_name'] = df['next_stop_id'].map(stop_id_to_name)

# Fill missing 'next_stop_id'
df['next_stop_id'].fillna('No next stop', inplace=True)

# Fill missing 'next_stop_name'
df['next_stop_name'].fillna('End of trip', inplace=True)

df.sort_values(['trip_id', 'stop_sequence'], inplace=True)
# setja flögg til að geta brugðist betur við.
df['is_first_stop'] = df.groupby('trip_id').cumcount() == 0
df['is_last_stop'] = df.groupby('trip_id').cumcount(ascending=False) == 0


# Display or further manipulate 'df'
display(df)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,direction_id,block_id,route_short_name,next_stop_id,next_arrival_time,next_stop_name,is_first_stop,is_last_stop
60,524766,13:27:00,13:27:00,90020295,1,Hlemmur B,64.143719,-21.915081,Hfj. Skarðshlíð,0,4134_1-G,1,90000052.0,13:28:00,Barónsstígur,True,False
153,524766,13:28:00,13:28:00,90000052,2,Barónsstígur,64.144475,-21.918978,Hfj. Skarðshlíð,0,4134_1-G,1,90000821.0,13:29:00,Bíó Paradís,False,False
246,524766,13:29:00,13:29:00,90000821,3,Bíó Paradís,64.145639,-21.924854,Hfj. Skarðshlíð,0,4134_1-G,1,90000054.0,13:30:00,Þjóðleikhúsið,False,False
339,524766,13:30:00,13:30:00,90000054,4,Þjóðleikhúsið,64.146842,-21.930941,Hfj. Skarðshlíð,0,4134_1-G,1,90000055.0,13:32:00,Lækjartorg A,False,False
432,524766,13:32:00,13:32:00,90000055,5,Lækjartorg A,64.147465,-21.936231,Hfj. Skarðshlíð,0,4134_1-G,1,90000056.0,13:33:00,Ráðhúsið,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285192,529211,15:01:00,15:01:00,90000061,31,Mýrargata,64.151683,-21.949006,Grandi,1,4409_14-F,14,90000062.0,15:03:00,Grandagarður,False,False
3285285,529211,15:03:00,15:03:00,90000062,32,Grandagarður,64.153109,-21.949784,Grandi,1,4409_14-F,14,90000063.0,15:04:00,Grunnslóð,False,False
3285378,529211,15:04:00,15:04:00,90000063,33,Grunnslóð,64.154464,-21.947669,Grandi,1,4409_14-F,14,90000064.0,15:05:00,Fiskislóð,False,False
3285471,529211,15:05:00,15:05:00,90000064,34,Fiskislóð,64.155849,-21.945205,Grandi,1,4409_14-F,14,90000020.0,15:06:00,Grandi,False,False


Búið til net sem samsvarar leiðarkerfinu á virkum degi. Sýnið fjölda hnúta og leggja í
netinu og finnið þá stoppistöð sem hefur hæstu útgráðuna (hvaða stöð og hver er útgráðan).
Netið sem þið búið til má vera hvernig sem er svo lengi sem það sé "pythonskt"í laginu.
Þ.e. við getum ítrað yfir hnúta í netinu og fyrir hvern hnút getum við ítrað yfir granna þess
hnúts. Útfærslan í Java í Algorithms bókinni uppfyllir þessi skilyrði og eftirfarandi einfaldi
python kóði gerir það líka.

```
G = {1: [2,3], 2: [], 3: [1,2] }
for v in G:
  print(u,':')
    for u in G[u,v]:
      print(u,' -> ',v)
```
Ef við viljum bæta við einhverjum upplýsingum þá er hægt að gera það í uppflettitöflu.
Ekki er leyfilegt að nota pakka fyrir net í python á borð við networkx eða sambærilegt.

#### Búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð

In [68]:
# búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð
all_stop_ids = df['stop_id'].unique()
stop_id_to_name = pd.Series(df.stop_name.values, index=df.stop_id).to_dict()

# print(stop_id_to_name[90000397])

#### Búum til netið G

In [32]:
#### hjálparföll fyrir netagerð
def convert_time_str_to_timedelta(time_str):
    parts = time_str.split(':')
    hours, minutes = int(parts[0]), int(parts[1])
    seconds = int(parts[2]) if len(parts) == 2 else 0
    days, hours = divmod(hours, 24)
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

def format_hours_to_time(hours):
    if hours is None:
        return None
    total_minutes = int(hours * 60)
    h, m = divmod(total_minutes, 60)
    return f"{h:02d}:{m:02d}"

def create_stop_name_to_ids_map(df):
    stop_name_to_ids = {}
    for index, row in df.iterrows():
        if row['stop_name'] not in stop_name_to_ids:
            stop_name_to_ids[row['stop_name']] = []
        stop_name_to_ids[row['stop_name']].append(row['stop_id'])
    return stop_name_to_ids

stop_name_to_ids = create_stop_name_to_ids_map(df)

In [85]:
G = {}
for _, row in df.iterrows():
    stop_id = row['stop_id']
    stop_name = row['stop_name']
    arrival_time = convert_time_str_to_timedelta(row['arrival_time'])
    departure_time = convert_time_str_to_timedelta(row['departure_time'])
    
    if not row['is_last_stop']:
        next_stop_id = row['next_stop_id']
        next_stop_name = row['next_stop_name']
        next_arrival_time = convert_time_str_to_timedelta(row['next_arrival_time'])
        # travel_time = next_arrival_time - arrival_time
        route_short_name = row['route_short_name']
        trip_headsign = row['trip_headsign']
        stop_lat = row['stop_lat']
        stop_lon = row['stop_lon']

        if stop_id not in G:
            G[stop_id] = []
        G[stop_id].append({
            'stop_name': stop_name,
            'route_short_name': route_short_name,
            'trip_headsign': trip_headsign,
            'arrival_time': arrival_time,
            'departure_time': departure_time,
            'next_stop_id': next_stop_id,
            'next_stop_name': next_stop_name,
            'next_arrival_time': next_arrival_time,
            'stop_lat': stop_lat,
            'stop_lon': stop_lon,
        })

In [61]:
# # til að prenta netið
# first_stop_name = next(iter(stop_name_to_ids))  # This retrieves the first stop name in the dictionary
# print(f"First Stop Name: {first_stop_name}")

# stop_ids = stop_name_to_ids[first_stop_name]
# # print(f"Stop IDs for '{first_stop_name}': {stop_ids}")

# count = 0
# max_display = 1  # Display only the first 10 stops for brevity
# for stop_id in stop_ids:
#     if stop_id in G and count < max_display:
#         print(f"Node: {stop_id}, Stop Name: {first_stop_name}, Latitude: {stop_lat}, Longitude: {stop_lon} ")
#         for edge in G[stop_id]:
#             print(f"  Edge to {edge['next_stop_name']} with id: {edge['next_stop_id']}")
#             print(f"    On its way to {edge['trip_headsign']}")
#             print(f"    Route: {edge['route_short_name']}")
#             print(f"    Arrival Time: {edge['arrival_time']}")
#             print(f"    Departure Time: {edge['departure_time']}")
#             print(f"    Arrival at next stop: {edge['next_arrival_time']}")
#         count += 1
#     elif count >= max_display:
#         print("Display limit reached.")
#         break

In [86]:
#### spurning hvort megi gera ráð fyrir báðum stoppum. fram og tilbaka. 
#### ég heyrði að lokatalan ætti að vera í kringum 1000 sem myndi meika sens. 2*538 ca 1000


#reikna útgráðu        
out_degrees = {stop_id: len(neighbors) for stop_id, neighbors in G.items()}

# Finna stoppistöð með hæstu útgráðu
max_out_degree_stop = max(out_degrees, key=out_degrees.get)
max_out_degree = out_degrees[max_out_degree_stop]
max_out_degree_stop_name = filtered_final_current_df[filtered_final_current_df['stop_id'] == max_out_degree_stop].iloc[0]['stop_name']

number_of_nodes = len(G)
print(f"Fjöldi hnúta er: {number_of_nodes}")

number_of_edges = sum(len(neighbors) for neighbors in G.values())
print(f"Fjöldi leggja er: {number_of_edges}")

print(f"Stoppistöð með hæstu útgráðu er {max_out_degree_stop_name} með útgráðu {max_out_degree}.")

Fjöldi hnúta er: 905
Fjöldi leggja er: 57977
Stoppistöð með hæstu útgráðu er Mjódd með útgráðu 538.


# 2.3.2 Leit (⋆⋆)
Útfærið leitarreikniritið á netinu eins og lýst var að ofan. Sýnið niðurstöðu leitarinnar þegar
við viljum komast frá Meistaravöllum (ath. það eru tvær stöðvar með þessu nafni) til FB,
kl 11:30.

In [87]:
def convert_time_str_to_timedelta(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return timedelta(hours=hours, minutes=minutes)

def format_hours_to_time(timedelta_obj):
    total_seconds = int(timedelta_obj.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, _ = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}"

def find_route(G, origin_id, destination_id, start_time):
    start_time = convert_time_str_to_timedelta(start_time)
    pq = []
    min_time = {stop_id: timedelta.max for stop_id in G}
    min_time[origin_id] = start_time
    previous_stop = {stop_id: None for stop_id in G}

    heapq.heappush(pq, (start_time, origin_id, []))  # Also push the path taken

    while pq:
        current_time, current_stop_id, path = heapq.heappop(pq)
        
        if current_stop_id == destination_id:
            return path + [(current_stop_id, current_time)], format_hours_to_time(current_time)
        
        for data in G[current_stop_id]:
            departure_time = data['departure_time']
            arrival_time = data['next_arrival_time']
            next_stop_id = data['next_stop_id']
            next_route = data['route_short_name']
            trip_headsign = data['trip_headsign']

            if current_time <= departure_time and arrival_time < min_time[next_stop_id]:
                min_time[next_stop_id] = arrival_time
                previous_stop[next_stop_id] = current_stop_id
                heapq.heappush(pq, (arrival_time, next_stop_id, path + [{'stop_id': current_stop_id, 'arrival_time': current_time, 'route': next_route, 'trip_headsign': trip_headsign}]))

            if path and path[-1]['route'] != next_route and current_time + timedelta(minutes=2) > departure_time:
                continue
    
    return None, "No route found"

meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')[0]
fb = find_unique_stop_ids(df, 'FB')[0]
route, arrival_time = find_route(G, meistaravellir, fb, '11:30')
if route:
    if isinstance(route[-1], dict):
        final_arrival_time = format_hours_to_time(route[-1]['arrival_time'])
    else:
        _, final_arrival_time = route[-1]
        final_arrival_time = format_hours_to_time(final_arrival_time)

    print(f"Komutími: {final_arrival_time}")


Komutími: 12:36



## 2.3.3 Framsetning (⋆)
Takið löglega leið og búið til leiðarlýsingu, þ.e. hvaða vagn á að taka klukkan hvað og hvar
á að skipta um stöð, svipað og gert er á myndinni að ofan.

In [90]:
meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')[0]
fb = find_unique_stop_ids(df, 'FB')[0]
route, arrival_time = find_route(G, meistaravellir, fb, '11:30')
if route:
    previous_route = None
    previous_stop_name = None
    previous_trip_headsign = None
    stop_count = 0
    start_stop_name = None

    for i, leg in enumerate(route):
        if isinstance(leg, dict) and 'stop_id' in leg and 'arrival_time' in leg and 'route' in leg:
            stop_id = leg['stop_id']
            arrival_time = leg['arrival_time']
            departure_time = leg.get('departure_time', arrival_time)
            current_route = leg['route'] 
            stop_name = stop_id_to_name[stop_id]
            trip_headsign = leg.get('trip_headsign', previous_trip_headsign)
            formatted_arrival_time = format_hours_to_time(arrival_time)
            formatted_departure_time = format_hours_to_time(departure_time)
        
            if previous_route is None:
                start_stop_name = stop_name  # Set the initial stop name
                previous_route = current_route
                previous_stop_name = stop_name
                previous_trip_headsign = trip_headsign
                stop_count = 1
            
                print(f"Byrjaðu á {stop_name} klukkan {formatted_departure_time}, á leið {current_route} til {trip_headsign}")

            elif current_route == previous_route:
                stop_count += 1

            else:
                if departure_time:
                    print(f"Frá {start_stop_name}, farðu {stop_count} stopp með leið {previous_route}, á leið til {previous_trip_headsign}")
                print(f"Skiptu yfir í strætó {current_route} á stoppistöð: {stop_name} á leið til {trip_headsign} klukkan {formatted_arrival_time}")
                start_stop_name = stop_name
                stop_count = 1  # reset stop count for new route
                previous_trip_headsign = trip_headsign
            
            previous_route = current_route
            previous_stop_name = stop_name

    # Print final arrival details
    if stop_count > 0 and start_stop_name:
        print(f"Frá {start_stop_name}, farðu {stop_count} stopp með leið {previous_route} á leið til {previous_trip_headsign}")
        if isinstance(route[-1], dict):
            final_stop_name = stop_id_to_name[route[+1]['stop_id']]
            final_arrival_time = format_hours_to_time(route[-1]['arrival_time'])
        elif isinstance(route[-1], tuple):
            final_stop_name =stop_id_to_name[route[-1][0]]
            final_arrival_time = format_hours_to_time(route[-1][1])
        else:
            final_stop_name = previous_stop_name
            final_arrival_time = format_hours_to_time(final_arrival_time)

        print(f"Komutími í {final_stop_name} : {final_arrival_time}")

Byrjaðu á Meistaravellir klukkan 00:00, á leið 13 til Eiðisgrandi
Frá Meistaravellir, farðu 27 stopp með leið 13, á leið til Eiðisgrandi
Skiptu yfir í strætó 11 á stoppistöð: RÚV á leið til Mjódd klukkan 07:31
Frá RÚV, farðu 8 stopp með leið 11, á leið til Mjódd
Skiptu yfir í strætó 3 á stoppistöð: Mjódd á leið til Sel/Fell klukkan 07:42
Frá Mjódd, farðu 1 stopp með leið 3, á leið til Sel/Fell
Skiptu yfir í strætó 12 á stoppistöð: Stekkjarbakki á leið til Mjódd/Ártún klukkan 07:52
Frá Stekkjarbakki, farðu 10 stopp með leið 12 á leið til Mjódd/Ártún
Komutími í FB : 08:03


## 2.3.4 Labb (⋆⋆)
Sumar stoppistöðvar eru nálægt hvor annari en ekki hægt að taka strætó á milli þeirra. Breytið netinu og reikniritinu ykkar til að höndla þetta tilfelli, notið GPS hnitin sem eru gefin í gögnunum og veljið hæfilega fjarlægð sem þið eruð til í að láta fólk labba. T.d. er hægt að labba úr 13 í Suðurveri yfir í 4 á Kringlumýrarbraut en 13 stoppar ekki þar. Sýnið dæmi um leið þar sem komutími styttist við að leyfa labb. Þessi aðferð við að tengja saman stöðvar er ekki fullkomin og sýnir að oft er mikilvægara að hafa aðgang að betri gögnum, t.d. eru stoppistöðvar nálægt í planinu en taka langan tíma að labba á milli.

In [120]:
def convert_time_str_to_timedelta(time_str):
    # Check if the input is already a timedelta object
    if isinstance(time_str, timedelta):
        return time_str

    # Handle cases where time might be NaN or None
    if pd.isna(time_str):
        return timedelta()

    # Split the string by colon and convert to integers
    try:
        hours, minutes, seconds = map(int, time_str.split(':'))
        return timedelta(hours=hours, minutes=minutes, seconds=seconds)
    except ValueError:
        # Handle potential errors in splitting or converting
        return timedelta()

def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth's radius in km
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance * 1000  # convert to meters

def add_connections_to_graph(G2, df):
    # Add bus connections
    for _, row in df.iterrows():
        stop_id = int(row['stop_id'])  # Convert to integer
        if stop_id not in G2:
            G2[stop_id] = []
        
        if not row['is_last_stop']:
            next_stop_id = int(row['next_stop_id'])  # Convert to integer
            G2[stop_id].append({
                'stop_name': row['stop_name'],
                'route_short_name': row['route_short_name'],
                'trip_headsign': row['trip_headsign'],
                'departure_time': convert_time_str_to_timedelta(row['departure_time']),
                'next_stop_id': next_stop_id,
                'next_stop_name': row['next_stop_name'],
                'next_arrival_time': convert_time_str_to_timedelta(row['next_arrival_time']),
                'stop_lat': row['stop_lat'],
                'stop_lon': row['stop_lon'],
                'travel_time': None,
                'walking': False
            })

    # Add walking connections
    stops = df[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates()
    walking_speed = 5 * 1000 / 3600  # Speed in m/s (5 km/h)
    for i, row1 in stops.iterrows():
        for j, row2 in stops.iterrows():
            if i != j and row1['stop_id'] != row2['stop_id'] and row1['stop_name'] != row2['stop_name'] :
                dist = calculate_distance(row1['stop_lat'], row1['stop_lon'], row2['stop_lat'], row2['stop_lon'])
                if dist <= 500:  # If the distance is less than or equal to 500 meters
                    walk_time_seconds = dist / walking_speed
                    walk_time = timedelta(seconds=walk_time_seconds)
                    G2[row1['stop_id']].append({
                        'stop_name': row['stop_name'],
                        'route_short_name': 'Walk',
                        'trip_headsign': f"Walk to {row2['stop_id']}",
                        'departure_time': None,  # No specific departure time for walking
                        'next_stop_id': row2['stop_id'],
                        'next_stop_name': row2['stop_name'],
                        'next_arrival_time': None,  # No specific arrival time for walking
                        'stop_lat': row2['stop_lat'],
                        'stop_lon': row2['stop_lon'],
                        'travel_time': walk_time,  # Here we assign the computed walk time
                        'walking': True
                    })

    return G2

G2 = {}
G2 = add_connections_to_graph(G2, df)

In [134]:
def print_stop_connections(G2, stop_name_to_ids, stop_name, max_display=10):
    if stop_name not in stop_name_to_ids:
        print(f"No stop IDs found for the stop name: {stop_name}")
        return

    stop_ids = stop_name_to_ids[stop_name]

    count = 0
    for stop_id in stop_ids:
        if stop_id in G2 and count < max_display:
            stop_lat, stop_lon = G2[stop_id][0]['stop_lat'], G2[stop_id][0]['stop_lon']
            print(f"Node: {stop_id}, Stop Name: {stop_name}, Latitude: {stop_lat}, Longitude: {stop_lon}")
            for edge in G2[stop_id]:
                transportation_mode = "Walking" if edge['walking'] else "Bus"
                travel_time = edge['travel_time'] if edge['walking'] else None
                # Print general edge information
                print(f"  Edge to {edge['next_stop_name']} with id: {edge['next_stop_id']}")
                print(f"    On its way to {edge['trip_headsign']}")
                print(f"    Route: {edge['route_short_name']}")
                # Conditional information based on transportation mode
                if edge['walking']:
                    print(f"    Transportation: {transportation_mode}")
                    print(f"    Travel time: {travel_time}")
                else:
                    print(f"    Departure Time: {edge['departure_time']}")
                    print(f"    Arrival at next stop: {edge['next_arrival_time']}")
            count += 1
        elif count >= max_display:
            print("Display limit reached.")
            break

# Usage example
# Make sure stop_name_to_ids is defined and G2 is your populated graph
# print_stop_connections(G2, stop_name_to_ids, 'Mjódd')
# print("Available stop IDs in G2:", G2.keys())


Available stop IDs in G2: dict_keys([90020295, 90000052, 90000821, 90000054, 90000055, 90000056, 90000296, 90000834, 90000845, 90000458, 90000460, 90000015, 10010802, 10000854, 13001201, 13001523, 13001589, 13001525, 14001526, 14001527, 14001528, 14001500, 14001561, 14001626, 14001542, 14001543, 14001544, 14001620, 14001612, 14001616, 14001617, 14001602, 14001660, 14001603, 14001604, 14001647, 14001648, 14001649, 14001696, 14001697, 14001638, 14001639, 14001640, 14001613, 14001614, 14001661, 14001615, 14001618, 14001619, 14001605, 14001610, 14001611, 14001587, 14001580, 14001541, 14001634, 14001597, 14001501, 14001502, 14001503, 13001504, 13001588, 13001506, 13001507, 10000856, 10040802, 90000016, 90000481, 90000483, 90000844, 90000835, 90000332, 90000003, 90000004, 90000001, 90000002, 90000822, 90000823, 90000030, 90000031, 90000032, 90000874, 90000224, 90000225, 90000422, 90000556, 90000423, 90000080, 90000081, 90000084, 90000268, 13001202, 10000826, 10000873, 10000875, 10000876, 100

In [133]:
def find_route(G, origin_id, destination_id, start_time):
    if origin_id not in G or destination_id not in G:
        return None, "Invalid origin or destination ID"
    
    # Convert start time from a string to a timedelta object
    start_time = convert_time_str_to_timedelta(start_time)
    # Priority queue to store the next node to visit
    pq = []
    # Dictionary to keep track of the minimum time to reach each node
    min_time = {stop_id: timedelta.max for stop_id in G}
    # Set the start node's minimum time as the start time
    min_time[origin_id] = start_time

    # Start the priority queue with the initial position
    heapq.heappush(pq, (start_time, origin_id, []))

    while pq:
        # Pop the node with the smallest arrival time
        current_time, current_stop_id, path = heapq.heappop(pq)
        
        # If we reach the destination, return the path and the total travel time
        if current_stop_id == destination_id:
            return path + [(current_stop_id, current_time)], format_hours_to_time(current_time)
        
        # Iterate through all possible next stops
        for data in G[current_stop_id]:
            if data['walking']:  # Skip walking routes if not needed
                continue

            next_stop_id = data['next_stop_id']
            next_departure_time = max(current_time, data['departure_time'])
            next_arrival_time = next_departure_time + data['next_arrival_time']

            # Check if the new path to this next stop is quicker than any previously found
            if next_arrival_time < min_time[next_stop_id]:
                min_time[next_stop_id] = next_arrival_time
                new_path = path + [{
                    'stop_id': current_stop_id,
                    'next_stop_id': next_stop_id,
                    'departure_time': next_departure_time,
                    'arrival_time': next_arrival_time,
                    'route': data['route_short_name'],
                    'trip_headsign': data['trip_headsign'],
                    'travel_mode': 'Bus'  # Only consider bus routes
                }]
                # Push the new path onto the queue
                heapq.heappush(pq, (next_arrival_time, next_stop_id, new_path))

    # If no route is found, return None
    return None, "No route found"

# Usage example
origin_id = '90000075'  # Replace with actual ID
destination_id = '90000068'  # Replace with actual ID
route, travel_time = find_route(G2, origin_id, destination_id, '11:30')

if route:
    for step in route:
        if isinstance(step, dict):
            print(f"From Stop ID {step['stop_id']} to Stop ID {step['next_stop_id']}, take {step['travel_mode']} via {step['route']}, departure at {format_hours_to_time(step['departure_time'])}, arrive by {format_hours_to_time(step['arrival_time'])}")
        else:
            _, arrival_time = step
            print(f"Arrive at destination Stop ID {step[0]} at {format_hours_to_time(arrival_time)}")
    print(f"Total travel time: {travel_time}")
else:
    print("No valid route found or invalid stop IDs.")

No valid route found or invalid stop IDs.


In [104]:
# def debug_print_graph(G, sample_size=5):
#     from itertools import islice
#     for stop_id, connections in islice(G.items(), sample_size):
#         print(f"Stop ID {stop_id} has connections:")
#         for conn in connections:
#             print(f"  to {conn['next_stop_id']} via {'Walk' if conn['walking'] else 'Bus'} with travel time {conn['next_arrival_time']}")
       
# debug_print_graph(G2, sample_size=5)

In [106]:
# def print_stop_connections(G2, stop_name_to_ids, stop_name, max_display=10):
#     if stop_name not in stop_name_to_ids:
#         print(f"No stop IDs found for the stop name: {stop_name}")
#         return

#     stop_ids = stop_name_to_ids[stop_name]

#     count = 0
#     for stop_id in stop_ids:
#         if stop_id in G2 and count < max_display:
#             stop_lat, stop_lon = G2[stop_id][0]['stop_lat'], G2[stop_id][0]['stop_lon']
#             print(f"Node: {stop_id}, Stop Name: {stop_name}, Latitude: {stop_lat}, Longitude: {stop_lon}")
#             for edge in G2[stop_id]:
#                 transportation_mode = "Walking" if edge['walking'] else "Bus"
#                 travel_time = edge['travel_time'] if edge['walking'] else None
#                 # Print general edge information
#                 print(f"  Edge to {edge['next_stop_name']} with id: {edge['next_stop_id']}")
#                 print(f"    On its way to {edge['trip_headsign']}")
#                 print(f"    Route: {edge['route_short_name']}")
#                 # Conditional information based on transportation mode
#                 if edge['walking']:
#                     print(f"    Transportation: {transportation_mode}")
#                     print(f"    Travel time: {travel_time}")
#                 else:
#                     print(f"    Departure Time: {edge['departure_time']}")
#                     print(f"    Arrival at next stop: {edge['next_arrival_time']}")
#             count += 1
#         elif count >= max_display:
#             print("Display limit reached.")
#             break

# # Usage example
# # Make sure stop_name_to_ids is defined and G2 is your populated graph
# # print_stop_connections(G2, stop_name_to_ids, 'Mjódd')


# def format_hours_to_time(timedelta_obj):
#     total_seconds = int(timedelta_obj.total_seconds())
#     hours, remainder = divmod(total_seconds, 3600)
#     minutes, _ = divmod(remainder, 60)
#     return f"{hours:02d}:{minutes:02d}"


# def print_route(route, final_arrival_time):
#     if not route:
#         print("No valid route found.")
#         return

#     print("Route Summary:")
#     for i, leg in enumerate(route):
#         if isinstance(leg, dict):
#             mode = leg['travel_mode']
#             stop_id = leg['stop_id']
#             next_stop_id = leg['next_stop_id']
#             route_name = leg['route']
#             trip_headsign = leg['trip_headsign']
#             departure_time = leg['departure_time']
#             arrival_time = leg['arrival_time']
#             formatted_departure_time = format_hours_to_time(departure_time)
#             formatted_arrival_time = format_hours_to_time(arrival_time)

#             travel_time = arrival_time - departure_time
#             formatted_travel_time = format_hours_to_time(travel_time)
            
#             # Print details based on whether it's a walk or a bus ride
#             if mode == "Walk":
#                 print(f"Walk from stop ID {stop_id_to_name[stop_id]} to stop ID {stop_id_to_name[next_stop_id]}, depart at {formatted_departure_time}, arrive by {formatted_arrival_time}, duration {formatted_travel_time}.")
#             else:
#                 print(f"Take bus route {route_name} to {trip_headsign} from stop ID {stop_id_to_name[stop_id]}, depart at {formatted_departure_time}, arrive by {formatted_arrival_time}, duration {formatted_travel_time}.")

#         elif isinstance(leg, tuple):  # This indicates arrival at the final destination
#             stop_id, arrival_time = leg
#             formatted_arrival_time = format_hours_to_time(arrival_time)
#             print(f"Arrive at destination stop ID {stop_id} at {formatted_arrival_time}.")

#     print(f"Total travel time: {final_arrival_time}")


# origin_id = find_unique_stop_ids(df, 'Meistaravellir')[0]
# destination_id = find_unique_stop_ids(df, 'FB')[0]
# start_time = '11:30'
# route, final_arrival_time = find_bus_walk_route(G, origin_id, destination_id, start_time)
# print_route(route, final_arrival_time)



Route Summary:
Take bus route 13 to Eiðisgrandi from stop ID Meistaravellir, depart at 07:00, arrive by 14:01, duration 07:01.
Take bus route 13 to Eiðisgrandi from stop ID KR, depart at 14:01, arrive by 21:03, duration 07:02.
Take bus route 11 to Seltjarnarnes from stop ID Kaplaskjól, depart at 21:03, arrive by 27:59, duration 06:56.
Take bus route 11 to Seltjarnarnes from stop ID Grænamýri, depart at 27:59, arrive by 34:56, duration 06:57.
Take bus route 13 to Eiðisgrandi from stop ID Skerjabraut, depart at 34:56, arrive by 42:03, duration 07:07.
Take bus route 13 to Sléttuvegur from stop ID Öldugrandi, depart at 42:03, arrive by 48:40, duration 06:37.
Take bus route 13 to Sléttuvegur from stop ID Seilugrandi, depart at 48:40, arrive by 55:18, duration 06:38.
Take bus route 13 to Sléttuvegur from stop ID Keilugrandi, depart at 55:18, arrive by 61:56, duration 06:38.
Take bus route 13 to Sléttuvegur from stop ID Boðagrandi, depart at 61:56, arrive by 68:35, duration 06:39.
Take bus ro

## 2.3.5 Tímamælingar (⋆)
Mælið fjölda fyrirspurna sem aðferðin ykkar getur svarað á sekúndu. Tryggið að mælingarnar blandi stuttum og löngum leiðum með ólíkum tímum.

## 2.3.6 Bestun og gagnagrindur (⋆⋆)
Notið niðurstöður úr tímamælingu til að ná betri afköstum. Hér skiptir máli hvernig framsetningin á netinu er framkvæmd, hvaða gagnagrindur eru notaðar og hvort hægt sé að bæta eitthvað.

## 2.3.7 Færri skiptingar (⋆⋆)
Í einhverjum tilfellum eru margar ólíkar leiðir sem taka jafnlangan tíma, þ.e. seinasti vagninn kemur á áfangastað á sama tíma en fjöldi vagna sem er tekinn er ekki sá sami. Útskýrið hvernig er hægt að breyta reikniritinu til skipta sem sjaldnast um vagn og útfærið þetta á netinu. Sýnið dæmi um strætóferð þar sem niðurstaðan breytist.

## 2.3.8 A(⋆⋆⋆)
Útfærið A∗ reikniritið (sjá kafla 10.8 í https://people.mpi-inf.mpg.de/~mehlhorn/ftp/
NewToolbox/spath.pdf) með því að velja heppilegt fjarlægðarfall f . Sýnið að fjarlægðar-
fallið sem er valið sé löglegt, þ.e. reikniritið skilar alltaf réttum niðurstöðum, og framkvæmið
tímamælingar sem mæla hversu miklu þessi breyting skilar.

## 2.3.9 Pokémon (⋆ ⋆ ⋆)
Finnið þá leið sem tekur mestan fjölda vagna. Athugið að þetta verkefni er mögulega NP-erfitt, takið eftir að netið eins og það er sett upp er ekki DAG (stefnt óhringað net) en það má breyta því í DAG með því að taka út hnútana sem samsvara stoppistöðvum og bæta við hnút fyrir stoppistöð og hvern tímapunkt (x, t), þá setjum við legg frá (x, t) til (x, t′) ef t′ er næsti tímapunktur fyrir þessa stöppistöð og legg sem fara frá (x, t) til v ef vagn fer frá x á tíma t.

Finnið, með einhverju móti, lengstu mögulegu leið sem tekur sem flesta ólíka strætis-
vagnaleiðir.